# Cover Type Prediction: Feature Engineering

Sébastien meyer

In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd
from numpy.random import MT19937, RandomState, SeedSequence

from scipy import stats
from scipy.cluster import hierarchy as hc

from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, LabelBinarizer, StandardScaler

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
font = {"size": 22}
mpl.rc("font", **font)

seed = 8005

np.random.seed(seed)
rs = RandomState(MT19937(SeedSequence(seed)))

In [ ]:
def plot_rep(df, feature):
    
    plt.figure(figsize=(10, 8))

    color = iter(plt.cm.Accent(np.linspace(0, 1, 7)))

    for i in range(1, 8):
    
        sns.kdeplot(df.loc[train_df["Cover_Type"] == i, feature], label=f"Cover_Type = {i}", color=next(color))

    plt.legend(loc="best")
    plt.title(f"Repartition of {feature} among Cover_Type")

    plt.show()
    
def plot_scatter(df, feat1, feat2):
    
    fig = plt.figure(figsize=(16,8))

    color = iter(plt.cm.Accent(np.linspace(0, 1, 7)))

    for i in range(1, 8):
    
        plt.scatter(
            train_df.loc[train_df["Cover_Type"] == i, feat1],
            train_df.loc[train_df["Cover_Type"] == i, feat2],
            color=next(color), s=100, label=f"Cover_Type = 1{i}"
        )

    plt.xlabel(feat1)
    plt.ylabel(feat2)
    plt.legend(loc="best")

    plt.show()

In [ ]:
# Read files
test_df = pd.read_csv("data/covtype.csv", index_col=["Id"])

training_ids = []

with open("data/training_ids.txt", "r", encoding="utf-8") as f:

    training_ids = f.read().split(",")
    training_ids = [int(x) for x in training_ids]

train_df = test_df.iloc[training_ids, :].copy()

# Eliminate useless variables
if "Soil_Type15" in train_df.columns:
    train_df.drop(columns=["Soil_Type15"], inplace=True)
    test_df.drop(columns=["Soil_Type15"], inplace=True)
    
# Binary variables and target
wild_var = [f"Wilderness_Area{i}" for i in range(1, 5)]
soil_var = [f"Soil_Type{i}" for i in range(1, 41) if i != 15]
label_var = ["Cover_Type"]

# Separate discrete and continuous variables
all_var = train_df.columns
disc_var = wild_var + soil_var + label_var
cont_var = [x for x in all_var if x not in disc_var]

## Hierarchical clustering

Hierarchical clustering allows to show features that are similar to each other for future interaction terms.

In [ ]:
corr = train_df.corr()
plt.figure(figsize=(20,20))

hc.dendrogram(
    hc.linkage(hc.distance.squareform(1-corr), method="average"), 
    labels=train_df.columns, orientation="left", leaf_font_size=14
)

plt.show()

In [ ]:
corr = np.around(stats.spearmanr(train_df).correlation, 2)

plt.figure(figsize=(20,20))

hc.dendrogram(
    hc.linkage(hc.distance.squareform(1-corr), method="average"), 
    labels=train_df.columns, orientation="left", leaf_font_size=14
)

plt.show()

## Missing values?

Sometimes, missing values do not appear as NaN or inf values, but rather as limit values set by the writers of the document. Let's see with an example.

In [ ]:
plot_scatter(train_df, "Hillshade_3pm", "Hillshade_Noon")

In [ ]:
etr_h3pm = ExtraTreesRegressor(
    n_estimators=238, max_depth=None, min_samples_split=2, min_samples_leaf=1,
    max_features="auto", max_leaf_nodes=None, min_impurity_decrease=5.76e-8,
    bootstrap=False, ccp_alpha=3.64e-6, random_state=seed, n_jobs=-1, verbose=0
)

h3pm_var = "Hillshade_3pm"
train_h3pm_pos = train_df.index[train_df[h3pm_var] != 0].tolist()
train_h3pm_zeros = train_df.index[train_df[h3pm_var] == 0].tolist()
test_h3pm_zeros = test_df.index[test_df[h3pm_var] == 0].tolist()

etr_h3pm.fit(train_df.drop(columns=[h3pm_var]).loc[train_h3pm_pos, :],
             train_df.loc[train_h3pm_pos, h3pm_var])

train_df.loc[train_h3pm_zeros, h3pm_var] = \
    etr_h3pm.predict(train_df.drop(columns=[h3pm_var]).loc[train_h3pm_zeros, :])
test_df.loc[test_h3pm_zeros, h3pm_var] = \
    etr_h3pm.predict(test_df.drop(columns=[h3pm_var]).loc[test_h3pm_zeros, :])

plot_scatter(train_df, "Hillshade_3pm", "Hillshade_Noon")

## Knowledge-domain features

### Transformations

Firstly, try with some combination or transformations: root, log, square, ratios, statistics, ...

In [ ]:
# Add ratio of distances to hydrology
train_df["Ratio_Distance_To_Hydrology"] = train_df["Vertical_Distance_To_Hydrology"]/train_df["Horizontal_Distance_To_Hydrology"]

# Add Log of distance to hydrology
train_df["Horizontal_Distance_To_Hydrology_Log"] = np.log(1+train_df["Horizontal_Distance_To_Hydrology"])

# Add Log of distance to fire points
train_df["Horizontal_Distance_To_Roadways_Log"] = np.log(1+train_df["Horizontal_Distance_To_Roadways"])

# Add Max of known values
train_df["Max"] = train_df.max(axis=1)

# Add Std of known values
train_df["Std"] = train_df.std(axis=1)

# There might be missing values for the ratio (0 horizontal distance)
imp = SimpleImputer(strategy="median")

train_df[["Ratio_Distance_To_Hydrology"]] = imp.fit_transform(train_df[["Ratio_Distance_To_Hydrology"]])

In [ ]:
plot_rep(train_df, "Ratio_Distance_To_Hydrology")
plot_rep(train_df, "Horizontal_Distance_To_Hydrology_Log")
plot_rep(train_df, "Horizontal_Distance_To_Roadways_Log")
plot_rep(train_df, "Max")
plot_rep(train_df, "Std")

### Signs

In [ ]:
train_df[cont_var].hist(figsize=(16, 12), bins=50)
plt.show()

Only some cover types have negative Vertical distance to hydrology, let's plot more information about this feature.

This feature does not seem to help us learn a lot of information, we will try to define a new feature around its positivity.

In [ ]:
train_df["Vertical_Distance_To_Hydrology_Sign"] = (train_df["Vertical_Distance_To_Hydrology"] > 0).astype(int)

plot_rep(train_df, "Vertical_Distance_To_Hydrology")
plot_rep(train_df, "Vertical_Distance_To_Hydrology_Sign")

By creating this feature, we help discriminating some of the cover types.

Take a look at the aspect: values are among 0 to 360. We could create two features out of this: shift Aspect to -180:180 and its sign.

In [ ]:
train_df["Shifted_Aspect"] = train_df["Aspect"] - 180
train_df["Shifted_Aspect_Sign"] = (train_df["Shifted_Aspect"] > 0).astype(int)

plot_rep(train_df, "Aspect")
plot_rep(train_df, "Shifted_Aspect_Sign")

Repartition of the signs are slightly different from cover types, but we might not learn as much information from this feature as with the sign of the vertical distance to hydrology.

Regarding distance to hydrology, we can also compute its total distance!

### Distances

In [ ]:
train_df["Distance_To_Hydrology"] = (train_df["Horizontal_Distance_To_Hydrology"].pow(2) + train_df["Vertical_Distance_To_Hydrology"].pow(2)).pow(0.5)

In [ ]:
plot_rep(train_df, "Horizontal_Distance_To_Hydrology")
plot_rep(train_df, "Vertical_Distance_To_Hydrology")
plot_rep(train_df, "Distance_To_Hydrology")

### Hillshade and angles

Total distance to hydrology is very similar to horizontal distance to hydrology, this might not help that much...

Now, we will look into Hillshades features, which have small correlation with the target.

In [ ]:
hillshades = ["Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm"]

train_df[hillshades].hist(figsize=(16, 12), bins=100)
plt.show()

In [ ]:
train_df["Angle_To_Hydrology"] = np.arctan(
    train_df["Vertical_Distance_To_Hydrology"]/train_df["Horizontal_Distance_To_Hydrology"]
)

plot_rep(train_df, "Angle_To_Hydrology")

In [ ]:
train_df["Mean_Hillshade"] = train_df[hillshades].sum(axis=1)/3
train_df["Aspect Hillshade_3pm"] = train_df["Aspect"] * train_df["Hillshade_3pm"]

In [ ]:
plot_rep(train_df, "Mean_Hillshade")
plot_rep(train_df, "Aspect Hillshade_3pm")

### Shifted distances

One of the most important features is that distances are related. Let's see with an example...

In [ ]:
fig = plt.figure(figsize=(16,8))

plt.scatter(
    train_df["Elevation"], train_df["Horizontal_Distance_To_Hydrology"],
    c=train_df.Cover_Type.values/7, s=100, cmap=plt.cm.Accent
)
plt.xlabel("Elevation")
plt.ylabel("Horizontal_Distance_To_Hydrology")

plt.show()

In [ ]:
fig = plt.figure(figsize=(16,8))

plt.scatter(
    train_df["Elevation"]-0.2*train_df["Horizontal_Distance_To_Hydrology"],
    train_df["Horizontal_Distance_To_Hydrology"],
    c=train_df.Cover_Type.values/7, s=100, cmap=plt.cm.Accent
)
plt.xlabel("Elevation")
plt.ylabel("Horizontal_Distance_To_Hydrology")

plt.show()

In [ ]:
plot_scatter(train_df, "Elevation", "Vertical_Distance_To_Hydrology")

In [ ]:
train_df["Elevation_Shifted_Vertical_Distance_To_Hydrology"] = train_df["Elevation"]-train_df["Vertical_Distance_To_Hydrology"]

plot_scatter(train_df, "Elevation_Shifted_Vertical_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology")

In [ ]:
fig = plt.figure(figsize=(16,8))

plt.scatter(
    train_df["Elevation"]-0.02*train_df["Horizontal_Distance_To_Roadways"],
    train_df["Horizontal_Distance_To_Roadways"],
    c=train_df.Cover_Type.values/7, s=100, cmap=plt.cm.Accent
)
plt.xlabel("Elevation")
plt.ylabel("Horizontal_Distance_To_Hydrology")

plt.show()

### Bins

Also, we can try to compute features that will help decision trees (boundaries!): plateaus, ...

In [ ]:
plt.hist(train_df["Elevation"], bins=20)

plt.show()

In [ ]:
label_names = [0, 1, 2]
cut_points = [1850, 2575, 3100, 3850]
train_df["Elevation_Plateau"] = pd.cut(train_df["Elevation"], cut_points, labels=[0, 1, 2]).astype(int)

In [ ]:
plot_rep(train_df, "Elevation_Plateau")

### Sum of categorical features

Finally, we might look at additions and substractions of some categorical features

In [ ]:
train_df["Soil_Type12_32"] = train_df["Soil_Type32"] + train_df["Soil_Type12"]
train_df["Soil_Type23_22_32_33"] = \
    train_df["Soil_Type23"] + train_df["Soil_Type22"] + train_df["Soil_Type32"] + train_df["Soil_Type33"]
train_df["Wilderness_Area1_plus_Soil_Type29"] = train_df["Wilderness_Area1"] + train_df["Soil_Type29"]
train_df["Wilderness_Area4_plus_Soil_Type3"] = train_df["Wilderness_Area4"] + train_df["Soil_Type3"]

In [ ]:
plot_rep(train_df, "Soil_Type12_32")
plot_rep(train_df, "Soil_Type23_22_32_33")
plot_rep(train_df, "Wilderness_Area1_plus_Soil_Type29")
plot_rep(train_df, "Wilderness_Area4_plus_Soil_Type3")

### Groups

From the description of the data set, we can observe that there exist groups of soil types: families & stonyness

In [ ]:
# Binary features
soil_var = [f"Soil_Type{i}" for i in range(1, 41) if i != 15]
wild_var = [f"Wilderness_Area{i}" for i in range(1, 5)]

# Add the variables on training data
s = train_df[wild_var].idxmax(axis=1).str[15:].astype(int) - 1
train_df["Wilderness_Area"] = s

s = train_df[soil_var].idxmax(axis=1).str[9:].astype(int) - 1
train_df["Soil_Type"] = s

# Stony/Rubly/Neither type of soil
stony_soil_types = [1, 2, 6, 9, 12, 18, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40]
rubly_soil_types = [3, 4, 5, 10, 11, 13]
other_soil_types = [7, 8, 14, 15, 16, 17, 19, 20, 21, 22]

stony_dict = {i: 1 if i in stony_soil_types else 0 for i in range(1, 41)}
rubly_dict = {i: 1 if i in rubly_soil_types else 0 for i in range(1, 41)}
other_dict = {i: 1 if i in other_soil_types else 0 for i in range(1, 41)}

train_df["Stony_Soil_Type"] = train_df["Soil_Type"].map(stony_dict)
train_df["Rubly_Soil_Type"] = train_df["Soil_Type"].map(rubly_dict)
train_df["Other_Soil_Type"] = train_df["Soil_Type"].map(other_dict)

In [ ]:
plot_rep(train_df, "Stony_Soil_Type")
plot_rep(train_df, "Rubly_Soil_Type")
plot_rep(train_df, "Other_Soil_Type")

In [ ]:
# Binary features
soil_var = [f"Soil_Type{i}" for i in range(1, 41) if i != 15]
wild_var = [f"Wilderness_Area{i}" for i in range(1, 5)]

# Add the variables on training data
s = train_df[soil_var].idxmax(axis=1).str[9:].astype(int)

# Families (only if more than one soil types are in)
ratake = [2, 4]
vanet = [2, 5, 6]
catamount = [10, 11, 13, 26, 31, 32, 33]
leighan = [21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 33, 38]
bullwark = [10, 11]
como = [29, 30]
moran = [38, 39, 40]
other = [3, 14, 15, 16, 19, 20, 34, 35, 37]

ratake_dict = {i: 1 if i in ratake else 0 for i in range(1, 41)}
vanet_dict = {i: 1 if i in vanet else 0 for i in range(1, 41)}
catamount_dict = {i: 1 if i in catamount else 0 for i in range(1, 41)}
leighan_dict = {i: 1 if i in leighan else 0 for i in range(1, 41)}
bullwark_dict = {i: 1 if i in bullwark else 0 for i in range(1, 41)}
como_dict = {i: 1 if i in como else 0 for i in range(1, 41)}
moran_dict = {i: 1 if i in moran else 0 for i in range(1, 41)}
other_dict = {i: 1 if i in other else 0 for i in range(1, 41)}

train_df["Ratake_Family_Soil_Type"] = s.map(ratake_dict)
train_df["Vanet_Family_Soil_Type"] = s.map(vanet_dict)
train_df["Catamount_Family_Soil_Type"] = train_df["Soil_Type"].map(catamount_dict)
train_df["Leighan_Family_Soil_Type"] = train_df["Soil_Type"].map(leighan_dict)
train_df["Bullwark_Family_Soil_Type"] = train_df["Soil_Type"].map(bullwark_dict)
train_df["Como_Family_Soil_Type"] = train_df["Soil_Type"].map(como_dict)
train_df["Moran_Family_Soil_Type"] = train_df["Soil_Type"].map(moran_dict)
train_df["Other_Family_Soil_Type"] = s.map(other_dict)

In [ ]:
plot_rep(train_df, "Bullwark_Family_Soil_Type")